In [1]:
import sys
from datetime import datetime
import json
from elasticsearch import Elasticsearch, exceptions as es_exceptions
from elasticsearch.helpers import scan
import pandas as pd

In [2]:
with open('config.json') as json_data:
    config = json.load(json_data,)

In [3]:
ind = 'xc-gstream'
KB=1024
MB=KB*1024
GB=MB*1024
TB=GB*1024

In [4]:
es = Elasticsearch(
    hosts=[{'host': config['ES_HOST'], 'scheme':'https', 'port':9200}],
    basic_auth=(config['ES_USER'], config['ES_PASS']),
    request_timeout=60)

if es.ping():
    print('connected to ES.')
else:
    print('no connection to ES.')
    sys.exit(1)

connected to ES.


In [5]:
start_int = '2022-06-01 00:00'
stop_int = '2022-12-21 00:00'
start_dt = datetime.strptime(start_int, '%Y-%m-%d %H:%M')
stop_dt = datetime.strptime(stop_int, '%Y-%m-%d %H:%M')
print('start:', start_dt, '\nstop: ', stop_dt)

start: 2021-06-12 00:00:00 
stop:  2021-06-21 00:00:00


In [6]:
query={
    "_source": ['host', 'access_cnt','lfn', 'size', 'b_bypass', 'b_hit', 'b_miss', 'n_blks', 'n_blks_done'],
    "query": {
        "range": {
            "@timestamp": {
                "gt": int(start_dt.timestamp()*1000),
                "lte": int(stop_dt.timestamp()*1000)
            }
        }
    }
}

In [7]:
data = {'access':[], 'host':[], 'lfn':[], 'scope':[], 'fn':[], 
'b_hit':[], 'b_miss':[], 'b_bypass':[], 'fsize':[], 'fill':[]}
docs_read = 0
scroll = scan(client=es, index=ind, query=query, timeout="5m")
for res in scroll:
    docs_read += 1
    if not docs_read % 50000:
        print('docs read', docs_read)
    # print(res)
    # break
    if not 'host' in res['_source']:
        print('bad document:', res)
        continue
    data['host'].append(res['_source']['host'])
    data['access'].append(res['_source']['access_cnt'])
    lfn=res['_source']['lfn'][12:]
    data['lfn'].append(lfn)
    data['scope'].append(lfn[:lfn.find('/')])
    data['fn'].append(lfn[lfn.rfind('/')+1:])
    data['b_hit'].append(res['_source']['b_hit'])
    data['b_miss'].append(res['_source']['b_miss'])
    data['b_bypass'].append(res['_source']['b_bypass'])
    data['fsize'].append(res['_source']['size'])
    data['fill'].append(res['_source']['n_blks_done']/res['_source']['n_blks'])
print('read:', docs_read)

docs read 20000
docs read 40000
docs read 60000
docs read 80000
docs read 100000
docs read 120000
docs read 140000
docs read 160000
docs read 180000
docs read 200000
docs read 220000
docs read 240000
docs read 260000
docs read 280000
docs read 300000
docs read 320000
docs read 340000
docs read 360000
docs read 380000
docs read 400000
docs read 420000
docs read 440000
docs read 460000
docs read 480000
docs read 500000
docs read 520000
docs read 540000
docs read 560000
docs read 580000
docs read 600000
docs read 620000
docs read 640000
docs read 660000
docs read 680000
docs read 700000
docs read 720000
docs read 740000
docs read 760000
docs read 780000
docs read 800000
docs read 820000
docs read 840000
docs read 860000
docs read 880000
docs read 900000
docs read 920000
docs read 940000
docs read 960000
read: 964278


In [9]:
df= pd.DataFrame.from_dict(data)
df.to_parquet('data/xcache_2022_II.parquet')  
df.head()


read perc     1.317715e-01
tot hit       1.389763e+02
tot miss      6.407506e+01
tot bypass    6.980863e+12
dtype: float64